# Asynchronous computing and parallel algorithms

* Asynchronous programming using std::async and std::future
* Asynchronous programming using std::async and std::future
* Experimental support of the parallel algorithms in GCC and MSVC
* Full support of parallel algorithms in HPX

## Asynchronous programming using std::async and std::future

In [10]:
%%writefile future.cpp
#include <future>
#include <iostream>

int add(int a, int b)
{
    return a + b;
}

int main(void)
{
int a = 5;
int b = 5;
std::future<int> result = std::async(add,a,b);

std::cout<< result.get() << std::endl;

return EXIT_SUCCESS;
}

file written: future.cpp

### Compilation

We need to add `-phtread` for the asyncronous execution

In [7]:
%%writefile example.cpp
#include <future>
#include <iostream>
#include <cstdlib>

int add(int a, int b)
{
    return a + b;
}

int main(void)
{

int a = 5;
int b = 5;

std::future<int> result = std::async(add,a,b);

std::cout<< result.get() << std::endl;

return EXIT_SUCCESS;
}

Overwriting example.cpp


In [9]:
!g++ example.cpp -pthread -o example

### Exectution

In [10]:
!./example

10


## Asynchronous programming using hpx::async and hpx::future

In [1]:
#include<run_hpx.cpp>
#include <hpx/future.hpp>
#include <iostream>

In [2]:
int add(int a, int b)
{
    return a + b;
}

In [3]:
run_hpx([](){

hpx::lcos::future<int> result = hpx::async(add,5,5);

std::cout << result.get() ;

});

10

(void) @0x7fc4222dbe60


###